In [ ]:
# default_exp main

# Processamento e Filtragem
> Módulo com funções que encapsulam o processamento completo e retorno de metadados, matrix de nível e estatísticas dos arquivos bin

In [ ]:
#hide 
#Standard Lib imports
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically
%config Completer.use_jedi = False

In [ ]:
#export
from datetime import datetime
from typing import *
import os
import logging
from fastcore.xtras import Path
from fastcore.script import call_parse, Param, store_true
from fastcore.basics import listify
from fastcore.foundation import L
import numpy as np
import pandas as pd
from rich.progress import Progress
from rich.console import Console
from rich.theme import Theme
from rich.logging import RichHandler
from rich import print
from rfpye.utils import *
from rfpye.constants import SPECTRAL_BLOCKS
from rfpye.parser import *
from loguru import logger

BTYPES = [21, 40] + SPECTRAL_BLOCKS
FILTER_ATTRS = {21: ['hostname', 'method'],
                40: ['altitude', 'latitude', 'longitude'],
                63: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'unit', 'description', 'start_mega', 'stop_mega'],
                67: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'unit', 'description', 'start_mega', 'stop_mega'],
                68: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'unit', 'description', 'start_mega', 'stop_mega']}

CACHE_FOLDER = Path.cwd() / ".cache"

In [ ]:
#exporti
logger.add("rfpye.log", rotation="1 week", compression='zip', backtrace=True, diagnose=True) 

2

In [ ]:
#exporti
custom_theme = Theme({"info": "dim cyan", "warning": "magenta", "danger": "bold red"})
console = Console(theme=custom_theme)

## Resumo Estatístico
A seguinte função recebe um DataFrame cujas linhas são as diferentes varreduras do espectro, cada uma com seu timestamp, e colunas as diferentes frequências centrais medidas. Essa função é chamada pela função `extract_bin_stats`

In [ ]:
#export
@logger.catch
def filter_spectrum(
    df: pd.DataFrame,
    time_start: str = None,
    time_stop: str = None,
    freq_start: str = None,
    freq_stop: str = None,
) -> pd.DataFrame:
    """Recebe o arquivo de espectro df e retorna de acordo com os filtros

    Args:
        df (pd.DataFrame): Arquivo de espectro. Timestamp como linhas e frequências como colunas
        time_start (str): Timestamp de início. Se None filtra desde o início do arquivo
        time_stop (str): Timestamp de fim. Se None filtra até o fim do arquivo
        freq_start (str): Filtro inicial de frequência. Se None retorna desde a menor frequências
        freq_stop (str): Filtro Final de frequência. Se None retorna até a maior frequência.

    Returns:
        pd.DataFrame: DataFrame com Frequência, min, max e mean após os filtros aplicados.
    """
    df = df.copy()
    if time_start is None:
        time_start = "01/01/2000"
    if time_stop is None:
        time_stop = "31/12/2100"
    try:
        time_start = pd.to_datetime(time_start)
        time_stop = pd.to_datetime(time_stop)
    except pd.errors.ParserError:
        log.error(
            f"[bold red blink] Datas inválidas! Verifique as strings de data {freq_start} e {freq_stop}"
        )

    try:
        df.set_index("index", inplace=True)
        df.index = pd.to_datetime(df.index)
    except pd.errors.KeyError:
        if not isinstance(df.index, pd.DatetimeIndex):
            log.warning(
                f"Não foi passado uma coluna ou índice com datetime a ser filtrado, todas as linhas serão processadas",
                exc_info=True,
            )
            time_start = 0
            time_stop = df.shape[0]

    cols = df.columns.values.astype("float")
    rows = df.index.values

    if freq_start is None:
        freq_start = 0
    if freq_stop is None:
        freq_stop = np.inf

    filtered_cols = df.columns[(float(freq_start) <= cols) & (cols <= float(freq_stop))]
    filtered_rows = df.index[(time_start <= rows) & (rows <= time_stop)]
    if len(filtered_cols) == 0 or len(filtered_rows) == 0:
        return None
    count = filtered_rows.shape[0]
    array = df.loc[filtered_rows, filtered_cols].values
    freq = filtered_cols.values.astype("float32")
    min_ = array.min(axis=0)
    max_ = array.max(axis=0)
    mean = array.mean(axis=0)
    return pd.DataFrame(
        {"Frequency": freq, "Min": min_, "Max": max_, "Mean": mean, "Count": count}
    )

In [ ]:
#exporti
def read_meta(filename):
    ext = filename.suffix
    if ext == ".csv":
        df = pd.read_csv(filename)
    elif ext == ".xlsx":
        df = pd.read_excel(filename, engine="openpyxl")
    elif ext == ".fth":
        df = pd.read_feather(filename)
        if "wallclock_datetime" in df.columns:
            df.set_index("wallclock_datetime", inplace=True)
    else:
        raise ValueError(f"Extension {ext} not implemented")
    return df

## Processamento e Extração 
A função a seguir é um wrapper de toda funcionalidade desta biblioteca. Ela recebe o caminho `entrada` para um arquivo `.bin` ou pasta contendo vários arquivos `.bin`, extrai os metadados e os dados de espectro. Mescla o timestamp dos metadados com o arquivo de espectro e salva ambos na pasta `saida`. Essa pasta é usada como repositório e cache dos dados processados que serão utilizados pela função `extract_bin_stats`.

In [ ]:
#export
def process_bin(
    entrada: str,
    saida: str,
    recursivo: bool = False,
    pastas: Iterable[str] = None,
    levels: bool = False,
    substituir: bool = False,
    dtype: str = "float16",
) -> None:
    """Recebe uma pasta ou arquivo bin, processa e salva os metadados e espectro na saida.

    Args:
        entrada (str): Caminho para a Pasta ou Arquivo .bin
        saida (str): Pasta onde salvar os arquivos processados
        recursivo (bool, optional): Buscar os arquivos de entrada recursivamente. Defaults to False.
        pastas (Iterable[str], optional): Limitar a busca a essas pastas. Defaults to None.
        levels (bool, optional): Extrair e salvar os dados de espectro. Defaults to False.
        substituir (bool, optional): Reprocessar arquivos já processados?. Defaults to False.
        dtype (str, optional): Tipo de dados a salvar o espectro. Defaults to "float16".
    """

    entrada = Path(entrada)
    if entrada.is_file():
        lista_bins = [entrada]
    else:
        lista_bins = get_files(
            entrada, extensions=[".bin"], recurse=recursivo, folders=pastas
        )
    parsed_bins = {}
    meta_path = Path(f"{saida}/meta")
    levels_path = Path(f"{saida}/levels")
    meta_path.mkdir(exist_ok=True, parents=True)
    levels_path.mkdir(exist_ok=True, parents=True)
    log_meta = Path(f"{saida}/log_meta.txt")
    log_levels = Path(f"{saida}/log_levels.txt")
    if substituir:
        done_meta = set()
        done_levels = set()
    else:

        done_meta = (
            set(log_meta.read_text().split("\n")) if log_meta.exists() else set()
        )
        done_levels = (
            set(log_levels.read_text().split("\n")) if log_levels.exists() else set()
        )

    console.rule("Lista de Arquivos a serem processados", style="bold red")
    console.print(
        [f.name for f in lista_bins],
        style="bold white",
        overflow="fold",
        justify="left",
    )
    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo .bin a processar :zzz:")
        return

    if not levels:
        lista_bins = [f for f in lista_bins if f.name not in done_meta]
    else:
        lista_bins = [f for f in lista_bins if f.name not in done_levels]

    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo novo a processar :zzz:")
        console.print(
            ":point_up: use --substituir no terminal ou substituir=True na chamada caso queira reprocessar os bins e sobrepôr os arquivos existentes :wink:"
        )
        return

    try:

        with Progress(transient=True, auto_refresh=False) as progress:
            bins = progress.track(
                lista_bins,
                total=len(lista_bins),
                description="[green]Processando Blocos Binários",
            )

            for file in bins:
                progress.console.print(f"[cyan]Processando Blocos de: [red]{file.name}")
                parsed_bins[file.name] = parse_bin(file)
                progress.refresh()

            lista_meta = [(k, v) for k, v in parsed_bins.items() if k not in done_meta]

            if lista_meta:
                blocks = progress.track(
                    lista_meta,
                    total=len(lista_meta),
                    description="[cyan]Exportando Metadados",
                )
                for filename, block_dict in blocks:
                    progress.console.print(f"[cyan]Extraindo Metadados de: [red]{file}")
                    export_metadata(filename, block_dict, meta_path, ext=".fth")
                    done_meta.add(file)
                    progress.refresh()
            if levels:
                lista_levels = lista_meta = [
                    (k, v) for k, v in parsed_bins.items() if k not in done_levels
                ]
                if lista_levels:
                    bins = progress.track(
                        lista_levels,
                        total=len(lista_levels),
                        description="[grey]Exportando Dados de Espectro",
                    )
                    for file, block_obj in bins:
                        progress.console.print(
                            f"[grey]Extraindo Espectro de: [red]{file}"
                        )
                        meta_index = []
                        blocks = block_obj["blocks"]
                        for (tipo, tid) in blocks.keys():
                            if tipo not in SPECTRAL_BLOCKS:
                                continue
                            meta_file = Path(
                                f"{meta_path}/{file}-B_{tipo}_TId_{tid}.fth"
                            )
                            if not meta_file.exists():
                                export_meta(
                                    file,
                                    block_obj,
                                    meta_path,
                                    ext=".fth",
                                )
                                done_meta.add(file)
                            meta_df = read_meta(meta_file)
                            meta_index.append(meta_df.index.tolist())
                        export_level(
                            file,
                            block_obj,
                            levels_path,
                            ext=".fth",
                            index=meta_index,
                            dtype=dtype,
                        )
                        done_levels.add(file)
                        progress.refresh()
        console.print("kbô :satisfied:")
    finally:
        log_meta.write_text("\n".join(sorted(list(done_meta))))
        log_levels.write_text("\n".join(sorted(list(done_levels))))

In [ ]:
#export
@logger.catch
def extract_bin_data(
    path: str,
    save_path: str = None,
    spec_data: bool = False,
    dtype: str = "float16",
) -> None:
    """Recebe uma pasta ou arquivo bin, processa e salva os metadados e espectro na saida.

    Args:
        path (str): Caminho para a Pasta ou Arquivo .bin
        saida (str): Pasta onde salvar os arquivos processados
        recursivo (bool, optional): Buscar os arquivos de entrada recursivamente. Defaults to False.
        pastas (Iterable[str], optional): Limitar a busca a essas pastas. Defaults to None.
        substituir (bool, optional): Reprocessar arquivos já processados?. Defaults to False.
        dtype (str, optional): Tipo de dados a salvar o espectro. Defaults to "float16".
    """

    path = Path(path)
    if path.is_file():
        lista_bins = [path]
    elif path.is_dir():
        lista_bins = get_files(
            path, extensions=[".bin"])
    else:
        raise ValueError(f"Caminho de Entrada inválido: {path}. Insira um caminho para uma pasta ou arquivo")

    parsed_bins = {}
    if save_path is not None:
        save_path = Path(save_path)
        save_path.mkdir(exist_ok=True, parents=True)
        log = Path(f"{save_path}/logger.txt")
        if substituir:
            done = set()
        else:

            done = (
                set(log.read_text().split("\n")) if log.exists() else set()
            )

    else:
        done = set()

    console.rule("Lista de Arquivos a serem processados", style="bold red")
    console.print(
        [f.name for f in lista_bins],
        style="bold white",
        overflow="fold",
        justify="left",
    )
    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo .bin a processar :zzz:")
        return

    lista_bins = [f for f in lista_bins if f.name not in done]

    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo novo a processar :zzz:")
        console.print(
            ":point_up: use --substituir no terminal ou substituir=True na chamada caso queira reprocessar os bins e sobrepôr os arquivos existentes :wink:"
        )
        return

    output = dict()
    for file in lista_bins:
        console.print(f"[green]Processando Blocos de: [red]{file.name}")
        parsed_bin = parse_bin(file, btypes=BTYPES)
        file_version, blocks = parsed_bin.values()
        metadata = export_metadata(blocks, filter_attrs=FILTER_ATTRS)
        out = dict()
        out['File_Name'] = file.name
        out['File_Version'] = file_version
        out['File_Type'] = 'RFEye Logger Trace'
        out['Device'] = 'Rfeye Node'
        out['Fluxos'] = dict()
        for (btype, tid), df in metadata.items():
            if btype == 21:
                out['Equipment_ID'] = df.hostname.item()
                out['Script_Version'] = df.method.item()
            elif btype == 40:
                out['Latitude'] = df.latitude.median()
                out['Longitude'] = df.longitude.median()
                out['Altitude'] = df.altitude.median()
                out['Count_GPS'] = df.shape[0]
                out['Sum_Latitude']  = df.latitude.sum()
                out['Sum_Longitude'] = df.longitude.sum()
            elif btype in SPECTRAL_BLOCKS:
                timestamp = df.index.values
                level = dict()
                level['Initial_Time'] = timestamp.min()
                level['Sample_Duration'] = df['sample'].median()
                fluxo = df.drop(['minimum', 'sample'], axis=1).iloc[0]
                level['Description'] = fluxo.description
                level['Start_Frequency'] = fluxo.start_mega
                level['Stop_Frequency'] = fluxo.stop_mega
                level['Trace_Type'] = fluxo.processing
                level['RBW'] = fluxo.bw
                level['Level_Units'] = fluxo.unit
                level['Num_Traces'] = df.shape[0]
                level['Vector_Length'] = fluxo.ndata
                level['Timestamp'] = timestamp
                if spec_data:
                    if save_path:
                        level['Minimum_Level'] = df.minimum.values.astype('float16')
                        level['Level_Data'] = extract_level(blocks[(btype, tid)], dtype=np.uint8)
                    else:
                        level['Frequency'] = np.linspace(fluxo.start_mega, fluxo.stop_mega, num=fluxo.ndata)
                        level['Level_Data'] = extract_level(blocks[(btype, tid)], dtype=dtype)


                out['Fluxos'][(btype, tid)] =  level
            else:
                print(btype)
        done.add(file.name)
        output[file.name] = out

    if save_path:
        log.write_text("\n".join(sorted(list(done))))
    return output

Uso mais comum: Diretório com arquivos CRFS bin
A função seguinte recebe um caminho para um arquivo `.bin` ou para uma pasta que contém arquivos bin e retorna os metadados do arquivo como um dicionário. 

In [ ]:
path = Path(r'D:\OneDrive - ANATEL\BinFiles\Combo3 (Tipos de dados 4, 7, 8, 60-65 e 67-69)')

In [ ]:
if not path.exists() or not len(get_files(path, extensions=['.bin'])):
    path = Path('.')
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --output-document 'rfeye002092_210208_T202310_CRFSBINv.5.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --output-document 'rfeye002092_210208_T203131_CRFSBINv.2.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --output-document 'rfeye002292_210208_T202215_CRFSBINv.4.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --output-document 'rfeye002292_210208_T203238_CRFSBINv.3.bin'


In [ ]:
dados = extract_bin_data(path)

─────────────────────────── Lista de Arquivos a serem processados ───────────────────────────

[                                                                                            
    'rfeye002092_210208_T202310_CRFSBINv.5.bin',                                             
    'rfeye002092_210208_T203131_CRFSBINv.2.bin',                                             
    'rfeye002292_210208_T202215_CRFSBINv.4.bin',                                             
    'rfeye002292_210208_T203238_CRFSBINv.3.bin'                                              
]                                                                                            

Processando Blocos de: rfeye002092_210208_T202310_CRFSBINv.5.bin

Processando Blocos de: rfeye002092_210208_T203131_CRFSBINv.2.bin

Processando Blocos de: rfeye002292_210208_T202215_CRFSBINv.4.bin

Processando Blocos de: rfeye002292_210208_T203238_CRFSBINv.3.bin

In [ ]:
for filename, dado in dados.items():
    print(f'Metadados do Arquivo {filename}')
    print({k:v for k,v in dado.items() if k != 'Fluxos'})
    print("Fluxos: ", {k:{i:j for i,j in fluxo.items()  if i not in ('Timestamp', 'Frequency', 'Level_Data')} for k,fluxo in dado['Fluxos'].items()})

Metadados do Arquivo rfeye002092_210208_T202310_CRFSBINv.5.bin

{
    'File_Name': 'rfeye002092_210208_T202310_CRFSBINv.5.bin',
    'File_Version': 23,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002092',
    'Script_Version': 'Script_CRFSBINv5',
    'Latitude': -12.826857,
    'Longitude': -38.478035,
    'Altitude': 124.3,
    'Count_GPS': 8,
    'Sum_Latitude': -102.61485,
    'Sum_Longitude': -307.82428
}

Fluxos: 
{
    (67, 20): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:10.500680000'),
        'Sample_Duration': 18073.0,
        'Description': 'ClearWrite. Peak.',
        'Start_Frequency': 76,
        'Stop_Frequency': 108,
        'Trace_Type': 'peak',
        'RBW': 7381,
        'Level_Units': 'dBm',
        'Num_Traces': 956,
        'Vector_Length': 8192
    },
    (67, 30): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:10.522721000'),
        'Sample_Duration': 8552.0,
        'Description': 'ClearWrite. Peak.',
        'Start_Frequency': 70,
        'Stop_Frequency': 110,
        'Trace_Type': 'peak',
        'RBW': 73828,
        'Level_Units': 'dBm',
        'Num_Traces': 988,
        'Vector_Length': 1024
    },
    (67, 10): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:11.979000000'),
        'Sample_Duration': 8076.0,
        'Description': 'ClearWrite. Peak.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 477,
        'Vector_Length': 3584
    },
    (67, 11): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:11.979000000'),
        'Sample_Duration': 8076.0,
        'Description': 'ClearWrite. Mean.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 477,
        'Vector_Length': 3584
    },
    (67, 12): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:20.801000000'),
        'Sample_Duration': 8081.0,
        'Description': 'MaxHold. Peak. Count = 10.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 26,
        'Vector_Length': 3584
    },
    (68, 14): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:20.801000000'),
        'Sample_Duration': 8081.0,
        'Description': 'Average. Mean. Count = 10.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 49,
        'Vector_Length': 3584
    },
    (67, 13): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:30.763000000'),
        'Sample_Duration': 8080.5,
        'Description': 'Average. Mean. Count = 10.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 32,
        'Vector_Length': 3584
    }
}

Metadados do Arquivo rfeye002092_210208_T203131_CRFSBINv.2.bin

{
    'File_Name': 'rfeye002092_210208_T203131_CRFSBINv.2.bin',
    'File_Version': 21,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node'
}

...

{
    'File_Name': 'rfeye002292_210208_T202215_CRFSBINv.4.bin',
    'File_Version': 22,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002292',
    'Script_Version': 'Script_CRFSBINv4',
    'Latitude': -12.971631,
    'Longitude': -38.48149,
    'Altitude': 151.65,
    'Count_GPS': 8,
    'Sum_Latitude': -103.773056,
    'Sum_Longitude': -307.85193
}

Fluxos: 
{
    (63, 20): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:15.500658000'),
        'Sample_Duration': 18140.0,
        'Description': 'Peak',
        'Start_Frequency': 76,
        'Stop_Frequency': 108,
        'Trace_Type': 'peak',
        'RBW': 3,
        'Level_Units': 'dBm',
        'Num_Traces': 1047,
        'Vector_Length': 8192
    },
    (63, 30): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:15.523638000'),
        'Sample_Duration': 8549.0,
        'Description': 'Peak',
        'Start_Frequency': 70,
        'Stop_Frequency': 110,
        'Trace_Type': 'peak',
        'RBW': 39,
        'Level_Units': 'dBm',
        'Num_Traces': 1048,
        'Vector_Length': 1024
    },
    (63, 10): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:16.457000000'),
        'Sample_Duration': 8083.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 524,
        'Vector_Length': 3584
    },
    (63, 11): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:16.457000000'),
        'Sample_Duration': 8083.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 524,
        'Vector_Length': 3584
    },
    (63, 12): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:25.618000000'),
        'Sample_Duration': 8076.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 52,
        'Vector_Length': 3584
    },
    (63, 13): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:25.618000000'),
        'Sample_Duration': 8076.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 52,
        'Vector_Length': 3584
    },
    (64, 14): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:25.618000000'),
        'Sample_Duration': 8076.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 48,
        'Level_Units': 'dBm',
        'Num_Traces': 52,
        'Vector_Length': 717
    }
}

Metadados do Arquivo rfeye002292_210208_T203238_CRFSBINv.3.bin

{
    'File_Name': 'rfeye002292_210208_T203238_CRFSBINv.3.bin',
    'File_Version': 21,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002292',
    'Script_Version': 'Script_CRFSBINv3',
    'Latitude': -12.971631,
    'Longitude': -38.481487,
    'Altitude': 150.6,
    'Count_GPS': 12,
    'Sum_Latitude': -155.65956,
    'Sum_Longitude': -461.7778
}

Fluxos: 
{}

In [ ]:
dados = extract_bin_data(path / 'rfeye002092_210208_T202310_CRFSBINv.5.bin', spec_data=True)

─────────────────────────── Lista de Arquivos a serem processados ───────────────────────────

['rfeye002092_210208_T202310_CRFSBINv.5.bin']                                                

Processando Blocos de: rfeye002092_210208_T202310_CRFSBINv.5.bin

In [ ]:
d = dados['rfeye002092_210208_T202310_CRFSBINv.5.bin']
fluxo = d['Fluxos'][67,20]
tempo = fluxo['Timestamp']
frequencias = np.linspace(fluxo['Start_Frequency'], fluxo['Stop_Frequency'], num=fluxo['Vector_Length'])
niveis = fluxo['Level_Data'] 
data = pd.DataFrame(niveis, index=tempo, columns=frequencias)
data.columns.name = "Frequencies"
data.index.name = "Time"
print(data.head())

[[-117.5 -113.  -111.5 ... -103.5 -110.   -97.5]
 [-113.5 -114.5 -111.  ...  -94.5  -98.5  -99.5]
 [-109.5 -111.5 -114.  ... -100.   -99.  -100.5]
 ...
 [-117.  -115.  -114.  ...  -99.5  -96.   -99. ]
 [-117.  -118.  -117.5 ...  -99.5  -93.5  -93.5]
 [-114.5 -110.5 -110.  ...  -96.5  -96.  -102. ]]

## Extrair dados estatísticos

In [ ]:
#exporti
def appended_mean(df: pd.Series) -> float:
    """Recebe um agrupamento do DataFrame e retorna sua média ponderada pela coluna Count

    Args:
        df (pd.DataFrame): Groupby do DataFrame

    Returns:
        float: Média Ponderada da linha pela coluna Count
    """
    return (df["Count"] * df["Mean"]).sum() / df["Count"].sum()

A função a seguir é a que será mais comumente chamada por outro módulo que utilizar esta lib. Ela recebe o caminho para um arquivo `.bin` e retorna um DataFrame com Frequência, Máximo, Mínimo e Média dos dados de Espectro presentes no arquivo `.bin`

In [ ]:
#export
def extract_bin_stats(
    filename: str,
    time_start: str = None,
    time_stop: str = None,
    freq_start: str = None,
    freq_stop: str = None,
    cache: str = CACHE_FOLDER,
) -> pd.DataFrame:
    """Recebe o caminho para um arquivo CRFS bin e retorna um dataframe com o resumo estatístico dos dados de espectro

    Args:
        filename (str): Caminho para o arquivo bin
        time_start (str): Timestamp de início. Se None filtra desde o início do arquivo
        time_stop (str): Timestamp de fim. Se None filtra até o fim do arquivo
        freq_start (str): Filtro inicial de frequência. Se None retorna desde a menor frequências
        freq_stop (str): Filtro Final de frequência. Se None retorna até a maior frequência.
        cache (str, optional): Caminho para a pasta de cache. Default é criar uma pasta oculta .cache no diretório atual.

    Returns:
        pd.DataFrame: Dataframe contendo o resumo estatístico do arquivo
    """

    cache = Path(cache)
    cache.mkdir(exist_ok=True, parents=True)
    filename = Path(filename)
    if filename.is_dir():
        filenames = get_files(filename, extensions=[".bin"])
    else:
        filenames = listify(filename)

    cached_files = get_files(cache / "levels")
    files = L()
    for filename in filenames:
        while True:
            # TODO filter based on metadata
            subset = cached_files.filter(lambda name: filename.stem in str(name))
            if not len(subset):
                process_bin(entrada=filename, saida=cache, levels=True)
            else:
                break
        files += subset
        subset = L()

    dfs = files.map(pd.read_feather)
    tids = files.map(lambda x: x.stem.split("_")[-1])
    spectra = dfs.map(
        filter_spectrum,
        time_start=time_start,
        time_stop=time_stop,
        freq_start=freq_start,
        freq_stop=freq_stop,
    )
    spectra = [(i, s) for i, s in zip(tids, spectra) if s is not None]
    columns = ["Tid", "Frequency", "Min", "Max", "Mean"]
    out = pd.DataFrame(columns=columns)
    if not spectra:
        log.warning(
            f"Os parâmetros repassados não correspondem a nenhum dado espectral do arquivo",
            exc_info=True,
        )
        return out
    for i, df in spectra:
        df["Tid"] = i
    spectra = [s for i, s in spectra]
    spectra = pd.concat(spectra)
    if len(spectra.Frequency) == len(spectra.Frequency.unique()):
        return spectra[columns]
    gb = spectra.groupby(["Tid", "Frequency"])
    out = gb.apply(appended_mean)
    Min = gb.min()["Min"]
    Max = gb.max()["Max"]
    Mean = gb.apply(appended_mean)
    out = pd.concat([Min, Max, Mean], axis=1).reset_index()
    out.columns = columns
    return out

Chamada da função somente fornecendo o caminho do arquivo `.bin`

Pela saída do código acima, vemos que o arquivo `.bin` foi processado, seus metadados e espectro extraídos e salvos. Como não passamos uma pasta de saída uma pasta local `.cache` é criada e os arquivos são salvos nela. Posteriormente o arquivo de espectro no cache é lido e o resumo estatístico das frequências no tempo é retornado.

Se chamarmos novamente a função com os mesmos argumentos, dessa vez a execução será mais rápida por conta do cache e assim o arquivo `.bin` não precisa ser processado novamente.

In [ ]:
dados = extract_bin_stats(binfile) ; dados

NameError: name 'extract_bin_stats' is not defined

Vemos que o arquivo possui frequências de 70MHz a 110MHz. Se tivermos interessados em faixas menos, podemos filtrá-las. Por exemplo, vamos filtrar pela faixa de FM somente `88 a 108`:

In [ ]:
dados = extract_bin_stats(binfile, freq_start=88, freq_stop=108) ; dados

NameError: name 'extract_bin_stats' is not defined

Para filtrarmos os dados estatísticos relativo a um tempo específico, precisamos saber de antemão qual o período específico o arquivo `.bin` compreende, se passarmos um período de tempo inválido, é retornado um DataFrame vazio e uma mensagem de aviso é salva no log.

In [ ]:
dados = extract_bin_stats(binfile, time_start='2021-05-21') ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
dados = extract_bin_stats(binfile, time_stop='2020-05-12') ; dados

NameError: name 'extract_bin_stats' is not defined

Esse arquivo específico compreende o período de `Timestamp('2020-12-01 15:34:21.578869') a Timestamp('2020-12-01 16:13:53.920250')`, um período de menos de uma hora.

Basta passarmos uma string de data válida, as horas, minutos e segundos são opcionais.

In [ ]:
dados = extract_bin_stats(binfile, time_start='2020-12-01 16:00') ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
dados = extract_bin_stats(binfile, time_start='01/12/2020 16:00') ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
dados = extract_bin_stats(binfile, time_stop='2020-12-01 16:00') ; dados

NameError: name 'extract_bin_stats' is not defined

Se quisermos filtrar para constar somente a faixa de FM e somente os 15 minutos de 15:45 a 16:00 do dia 01/12/2020 

In [ ]:
dados = extract_bin_stats(binfile, 
                          time_start='01/12/2020 15:45',
                          time_stop='2020-12-01 16:00',
                          freq_start=88,
                          freq_stop=108) 
dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
Converted Untitled.ipynb.
